# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_id_text = []
    for sentence in source_text.split('\n'):
        sentence_list = []
        for word in sentence.split():
            sentence_list.append(source_vocab_to_int[word])
        source_id_text.append(sentence_list)
    target_id_text = []
    for sentence in target_text.split('\n'):
        sentence_list = []
        for word in sentence.split():
            sentence_list.append(target_vocab_to_int[word])
        sentence_list.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(sentence_list)
    
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1


C:\Anaconda\lib\site-packages\ipykernel\__main__.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [9]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    input_text = tf.placeholder(tf.int32, [None, None], name = "input")
    targets = tf.placeholder(tf.int32, [None, None], name = "targets")
    learning_rate = tf.placeholder(tf.float32, name = "learning_rate")
    keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
    target_seq_length = tf.placeholder(tf.int32, [None], name = "target_sequence_length")
    max_target_seq_len = tf.reduce_max(target_seq_length, name = "max_target_len")
    source_seq_length = tf.placeholder(tf.int32, [None], name ="source_sequence_length")
    return (input_text, targets, learning_rate, keep_prob, target_seq_length, max_target_seq_len, source_seq_length)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "C:\\Anaconda\\lib\\runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "C:\\Anaconda\\lib\\runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "C:\\Anaconda\\lib\\site-packages\\ipykernel\\__main__.py", line 3, in <module>\n    app.launch_new_instance()', 'File "C:\\Anaconda\\lib\\site-packages\\traitlets\\config\\application.py", line 658, in launch_instance\n    app.start()', 'File "C:\\Anaconda\\lib\\site-packages\\ipykernel\\kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "C:\\Anaconda\\lib\\site-packages\\zmq\\eventloop\\ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'File "C:\\Anaconda\\lib\\sit

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [10]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    sliced_target_data = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    id_to_be_added = tf.fill([batch_size, 1], target_vocab_to_int['<GO>'])
    decoder_inp = tf.concat([id_to_be_added, sliced_target_data], 1)
    return decoder_inp

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [11]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    embedded_encoder = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                                        vocab_size = source_vocab_size, 
                                                        embed_dim = encoding_embedding_size)
    def build_lstm_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_with_dropout = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
        return lstm_with_dropout
    stacked_cell = []
    for i in range(num_layers):
        stacked_cell.append(build_lstm_cell(rnn_size, keep_prob))
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(stacked_cell)
    rnn_output, rnn_state = tf.nn.dynamic_rnn(stacked_lstm, embedded_encoder, source_sequence_length, dtype = tf.float32)
    return (rnn_output, rnn_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
    final_output,_,_ = tf.contrib.seq2seq.dynamic_decode(basic_decoder, maximum_iterations = max_summary_length)
    return final_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [16]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_ids = tf.tile([start_of_sequence_id], [batch_size])
    greedy_embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_ids, end_of_sequence_id)
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, greedy_embedding_helper, encoder_state, output_layer)
    final_output,_,_ = tf.contrib.seq2seq.dynamic_decode(basic_decoder, maximum_iterations = max_target_sequence_length)
    return final_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [17]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    def build_lstm_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_with_dropout = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
        return lstm_with_dropout
    stacked_cell = []
    for i in range(num_layers):
        stacked_cell.append(build_lstm_cell(rnn_size, keep_prob))
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(stacked_cell)
    
    decoder_output_dense_layer = Dense(target_vocab_size, 
                                       kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev = 0.1))
    
    with tf.variable_scope('decoding_layer') as scope:
        training_decoder_output = decoding_layer_train(encoder_state, stacked_lstm, dec_embed_input, 
                         target_sequence_length, max_target_sequence_length, 
                         decoder_output_dense_layer, keep_prob)
        scope.reuse_variables()
        inference_decoder_output = decoding_layer_infer(encoder_state, stacked_lstm, dec_embeddings, 
                                                        target_vocab_to_int['<GO>'],target_vocab_to_int['<EOS>'], 
                                                        max_target_sequence_length, target_vocab_size, 
                                                        decoder_output_dense_layer, batch_size, keep_prob)
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [19]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    encoder_output, encoder_out_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                                                       source_sequence_length, source_vocab_size, enc_embedding_size)
    processed_inp_to_decoder = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    training_decoder_output, inference_decoder_output = decoding_layer(processed_inp_to_decoder, encoder_out_state,
                                                                       target_sequence_length, max_target_sentence_length,
                                                                       rnn_size, num_layers, 
                                                                       target_vocab_to_int, target_vocab_size,
                                                                       batch_size, keep_prob, dec_embedding_size)
    return (training_decoder_output, inference_decoder_output)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [24]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.6
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/538 - Train Accuracy: 0.2436, Validation Accuracy: 0.3278, Loss: 4.8710
Epoch   0 Batch    2/538 - Train Accuracy: 0.2869, Validation Accuracy: 0.3473, Loss: 4.2009
Epoch   0 Batch    3/538 - Train Accuracy: 0.2658, Validation Accuracy: 0.3469, Loss: 3.9839
Epoch   0 Batch    4/538 - Train Accuracy: 0.2725, Validation Accuracy: 0.3473, Loss: 3.6920
Epoch   0 Batch    5/538 - Train Accuracy: 0.3125, Validation Accuracy: 0.3627, Loss: 3.4949
Epoch   0 Batch    6/538 - Train Accuracy: 0.3181, Validation Accuracy: 0.3596, Loss: 3.3779
Epoch   0 Batch    7/538 - Train Accuracy: 0.3109, Validation Accuracy: 0.3732, Loss: 3.3824
Epoch   0 Batch    8/538 - Train Accuracy: 0.3023, Validation Accuracy: 0.3651, Loss: 3.3224
Epoch   0 Batch    9/538 - Train Accuracy: 0.3141, Validation Accuracy: 0.3752, Loss: 3.3735
Epoch   0 Batch   10/538 - Train Accuracy: 0.3115, Validation Accuracy: 0.3865, Loss: 3.2433
Epoch   0 Batch   11/538 - Train Accuracy: 0.3352, Validation Accuracy

Epoch   0 Batch   90/538 - Train Accuracy: 0.5166, Validation Accuracy: 0.5339, Loss: 1.2045
Epoch   0 Batch   91/538 - Train Accuracy: 0.5037, Validation Accuracy: 0.5415, Loss: 1.2101
Epoch   0 Batch   92/538 - Train Accuracy: 0.4877, Validation Accuracy: 0.5323, Loss: 1.1864
Epoch   0 Batch   93/538 - Train Accuracy: 0.4900, Validation Accuracy: 0.5344, Loss: 1.1777
Epoch   0 Batch   94/538 - Train Accuracy: 0.5080, Validation Accuracy: 0.5336, Loss: 1.1813
Epoch   0 Batch   95/538 - Train Accuracy: 0.4885, Validation Accuracy: 0.4881, Loss: 1.0506
Epoch   0 Batch   96/538 - Train Accuracy: 0.4901, Validation Accuracy: 0.5020, Loss: 1.0560
Epoch   0 Batch   97/538 - Train Accuracy: 0.4973, Validation Accuracy: 0.5353, Loss: 1.1056
Epoch   0 Batch   98/538 - Train Accuracy: 0.5257, Validation Accuracy: 0.5421, Loss: 1.0328
Epoch   0 Batch   99/538 - Train Accuracy: 0.4693, Validation Accuracy: 0.5394, Loss: 1.0905
Epoch   0 Batch  100/538 - Train Accuracy: 0.5303, Validation Accuracy

Epoch   0 Batch  179/538 - Train Accuracy: 0.5887, Validation Accuracy: 0.6126, Loss: 0.7491
Epoch   0 Batch  180/538 - Train Accuracy: 0.6282, Validation Accuracy: 0.6204, Loss: 0.7117
Epoch   0 Batch  181/538 - Train Accuracy: 0.5736, Validation Accuracy: 0.6151, Loss: 0.7590
Epoch   0 Batch  182/538 - Train Accuracy: 0.5596, Validation Accuracy: 0.6088, Loss: 0.7423
Epoch   0 Batch  183/538 - Train Accuracy: 0.6196, Validation Accuracy: 0.6154, Loss: 0.6894
Epoch   0 Batch  184/538 - Train Accuracy: 0.5915, Validation Accuracy: 0.6126, Loss: 0.6912
Epoch   0 Batch  185/538 - Train Accuracy: 0.6018, Validation Accuracy: 0.6069, Loss: 0.7217
Epoch   0 Batch  186/538 - Train Accuracy: 0.6014, Validation Accuracy: 0.6023, Loss: 0.7128
Epoch   0 Batch  187/538 - Train Accuracy: 0.6071, Validation Accuracy: 0.5946, Loss: 0.6814
Epoch   0 Batch  188/538 - Train Accuracy: 0.5629, Validation Accuracy: 0.5985, Loss: 0.7194
Epoch   0 Batch  189/538 - Train Accuracy: 0.6018, Validation Accuracy

Epoch   0 Batch  268/538 - Train Accuracy: 0.6494, Validation Accuracy: 0.6465, Loss: 0.5529
Epoch   0 Batch  269/538 - Train Accuracy: 0.6346, Validation Accuracy: 0.6527, Loss: 0.5729
Epoch   0 Batch  270/538 - Train Accuracy: 0.6367, Validation Accuracy: 0.6554, Loss: 0.5807
Epoch   0 Batch  271/538 - Train Accuracy: 0.6402, Validation Accuracy: 0.6573, Loss: 0.5842
Epoch   0 Batch  272/538 - Train Accuracy: 0.6223, Validation Accuracy: 0.6612, Loss: 0.6102
Epoch   0 Batch  273/538 - Train Accuracy: 0.6447, Validation Accuracy: 0.6614, Loss: 0.5873
Epoch   0 Batch  274/538 - Train Accuracy: 0.6037, Validation Accuracy: 0.6559, Loss: 0.6176
Epoch   0 Batch  275/538 - Train Accuracy: 0.6230, Validation Accuracy: 0.6557, Loss: 0.5985
Epoch   0 Batch  276/538 - Train Accuracy: 0.6533, Validation Accuracy: 0.6602, Loss: 0.5929
Epoch   0 Batch  277/538 - Train Accuracy: 0.6420, Validation Accuracy: 0.6619, Loss: 0.5700
Epoch   0 Batch  278/538 - Train Accuracy: 0.6459, Validation Accuracy

Epoch   0 Batch  357/538 - Train Accuracy: 0.6836, Validation Accuracy: 0.6871, Loss: 0.4793
Epoch   0 Batch  358/538 - Train Accuracy: 0.6721, Validation Accuracy: 0.6903, Loss: 0.4830
Epoch   0 Batch  359/538 - Train Accuracy: 0.6879, Validation Accuracy: 0.6886, Loss: 0.4606
Epoch   0 Batch  360/538 - Train Accuracy: 0.6680, Validation Accuracy: 0.6928, Loss: 0.4827
Epoch   0 Batch  361/538 - Train Accuracy: 0.6914, Validation Accuracy: 0.6955, Loss: 0.4613
Epoch   0 Batch  362/538 - Train Accuracy: 0.6968, Validation Accuracy: 0.6914, Loss: 0.4510
Epoch   0 Batch  363/538 - Train Accuracy: 0.6793, Validation Accuracy: 0.6911, Loss: 0.4516
Epoch   0 Batch  364/538 - Train Accuracy: 0.6463, Validation Accuracy: 0.6926, Loss: 0.4981
Epoch   0 Batch  365/538 - Train Accuracy: 0.6840, Validation Accuracy: 0.6930, Loss: 0.4561
Epoch   0 Batch  366/538 - Train Accuracy: 0.6908, Validation Accuracy: 0.6951, Loss: 0.4839
Epoch   0 Batch  367/538 - Train Accuracy: 0.6986, Validation Accuracy

...

Epoch   9 Batch  161/538 - Train Accuracy: 0.9439, Validation Accuracy: 0.9373, Loss: 0.0527
Epoch   9 Batch  162/538 - Train Accuracy: 0.9475, Validation Accuracy: 0.9347, Loss: 0.0517
Epoch   9 Batch  163/538 - Train Accuracy: 0.9386, Validation Accuracy: 0.9329, Loss: 0.0629
Epoch   9 Batch  164/538 - Train Accuracy: 0.9402, Validation Accuracy: 0.9256, Loss: 0.0498
Epoch   9 Batch  165/538 - Train Accuracy: 0.9600, Validation Accuracy: 0.9260, Loss: 0.0474
Epoch   9 Batch  166/538 - Train Accuracy: 0.9578, Validation Accuracy: 0.9263, Loss: 0.0554
Epoch   9 Batch  167/538 - Train Accuracy: 0.9342, Validation Accuracy: 0.9284, Loss: 0.0717
Epoch   9 Batch  168/538 - Train Accuracy: 0.9270, Validation Accuracy: 0.9313, Loss: 0.0624
Epoch   9 Batch  169/538 - Train Accuracy: 0.9600, Validation Accuracy: 0.9395, Loss: 0.0478
Epoch   9 Batch  170/538 - Train Accuracy: 0.9459, Validation Accuracy: 0.9416, Loss: 0.0539
Epoch   9 Batch  171/538 - Train Accuracy: 0.9363, Validation Accuracy

Epoch   9 Batch  250/538 - Train Accuracy: 0.9525, Validation Accuracy: 0.9345, Loss: 0.0434
Epoch   9 Batch  251/538 - Train Accuracy: 0.9514, Validation Accuracy: 0.9354, Loss: 0.0465
Epoch   9 Batch  252/538 - Train Accuracy: 0.9554, Validation Accuracy: 0.9361, Loss: 0.0391
Epoch   9 Batch  253/538 - Train Accuracy: 0.9407, Validation Accuracy: 0.9458, Loss: 0.0534
Epoch   9 Batch  254/538 - Train Accuracy: 0.9467, Validation Accuracy: 0.9487, Loss: 0.0640
Epoch   9 Batch  255/538 - Train Accuracy: 0.9609, Validation Accuracy: 0.9505, Loss: 0.0474
Epoch   9 Batch  256/538 - Train Accuracy: 0.9490, Validation Accuracy: 0.9517, Loss: 0.0600
Epoch   9 Batch  257/538 - Train Accuracy: 0.9661, Validation Accuracy: 0.9576, Loss: 0.0533
Epoch   9 Batch  258/538 - Train Accuracy: 0.9559, Validation Accuracy: 0.9547, Loss: 0.0533
Epoch   9 Batch  259/538 - Train Accuracy: 0.9676, Validation Accuracy: 0.9569, Loss: 0.0464
Epoch   9 Batch  260/538 - Train Accuracy: 0.9195, Validation Accuracy

Epoch   9 Batch  339/538 - Train Accuracy: 0.9555, Validation Accuracy: 0.9542, Loss: 0.0487
Epoch   9 Batch  340/538 - Train Accuracy: 0.9436, Validation Accuracy: 0.9554, Loss: 0.0555
Epoch   9 Batch  341/538 - Train Accuracy: 0.9395, Validation Accuracy: 0.9576, Loss: 0.0471
Epoch   9 Batch  342/538 - Train Accuracy: 0.9381, Validation Accuracy: 0.9561, Loss: 0.0492
Epoch   9 Batch  343/538 - Train Accuracy: 0.9641, Validation Accuracy: 0.9565, Loss: 0.0571
Epoch   9 Batch  344/538 - Train Accuracy: 0.9609, Validation Accuracy: 0.9515, Loss: 0.0547
Epoch   9 Batch  345/538 - Train Accuracy: 0.9483, Validation Accuracy: 0.9466, Loss: 0.0533
Epoch   9 Batch  346/538 - Train Accuracy: 0.9301, Validation Accuracy: 0.9458, Loss: 0.0674
Epoch   9 Batch  347/538 - Train Accuracy: 0.9400, Validation Accuracy: 0.9446, Loss: 0.0561
Epoch   9 Batch  348/538 - Train Accuracy: 0.9360, Validation Accuracy: 0.9455, Loss: 0.0493
Epoch   9 Batch  349/538 - Train Accuracy: 0.9625, Validation Accuracy

Epoch   9 Batch  428/538 - Train Accuracy: 0.9624, Validation Accuracy: 0.9592, Loss: 0.0411
Epoch   9 Batch  429/538 - Train Accuracy: 0.9496, Validation Accuracy: 0.9569, Loss: 0.0556
Epoch   9 Batch  430/538 - Train Accuracy: 0.9428, Validation Accuracy: 0.9583, Loss: 0.0569
Epoch   9 Batch  431/538 - Train Accuracy: 0.9406, Validation Accuracy: 0.9593, Loss: 0.0502
Epoch   9 Batch  432/538 - Train Accuracy: 0.9434, Validation Accuracy: 0.9608, Loss: 0.0574
Epoch   9 Batch  433/538 - Train Accuracy: 0.9424, Validation Accuracy: 0.9563, Loss: 0.0785
Epoch   9 Batch  434/538 - Train Accuracy: 0.9371, Validation Accuracy: 0.9565, Loss: 0.0491
Epoch   9 Batch  435/538 - Train Accuracy: 0.9492, Validation Accuracy: 0.9545, Loss: 0.0501
Epoch   9 Batch  436/538 - Train Accuracy: 0.9521, Validation Accuracy: 0.9560, Loss: 0.0580
Epoch   9 Batch  437/538 - Train Accuracy: 0.9492, Validation Accuracy: 0.9569, Loss: 0.0569
Epoch   9 Batch  438/538 - Train Accuracy: 0.9531, Validation Accuracy

Epoch   9 Batch  517/538 - Train Accuracy: 0.9479, Validation Accuracy: 0.9435, Loss: 0.0410
Epoch   9 Batch  518/538 - Train Accuracy: 0.9373, Validation Accuracy: 0.9423, Loss: 0.0565
Epoch   9 Batch  519/538 - Train Accuracy: 0.9568, Validation Accuracy: 0.9435, Loss: 0.0518
Epoch   9 Batch  520/538 - Train Accuracy: 0.9473, Validation Accuracy: 0.9396, Loss: 0.0508
Epoch   9 Batch  521/538 - Train Accuracy: 0.9592, Validation Accuracy: 0.9442, Loss: 0.0606
Epoch   9 Batch  522/538 - Train Accuracy: 0.9480, Validation Accuracy: 0.9458, Loss: 0.0449
Epoch   9 Batch  523/538 - Train Accuracy: 0.9549, Validation Accuracy: 0.9476, Loss: 0.0481
Epoch   9 Batch  524/538 - Train Accuracy: 0.9533, Validation Accuracy: 0.9476, Loss: 0.0488
Epoch   9 Batch  525/538 - Train Accuracy: 0.9487, Validation Accuracy: 0.9490, Loss: 0.0561
Epoch   9 Batch  526/538 - Train Accuracy: 0.9555, Validation Accuracy: 0.9490, Loss: 0.0516
Epoch   9 Batch  527/538 - Train Accuracy: 0.9455, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [30]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    word_ids = []
    sentence = sentence.lower()
    for word in sentence.split():
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [31]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [158, 38, 15, 201, 227, 29, 32]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [286, 278, 268, 222, 132, 178, 352, 202, 1]
  French Words: il a vu un vieux camion vert . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.